# Using LHS Value to Debug Unsolved Problems

This notebook aims to show how to use the left-hand side (LHS) value (`Constraint.e`) to debug unsolved problems.

The LHS value is the value of the left-hand side of the equation. It is useful to check which constraints are violated.

In [1]:
import ams

Generating code for 1 models on 12 processes.


In [2]:
ams.__version__

'0.9.12rc1.post5.dev0+gc3478b86'

In [3]:
ams.config_logger(stream_level=20)

First, let's solve a DCOPF.

In [4]:
sp = ams.load(ams.get_case('matpower/case14.m'))

Parsing input file "/Users/jinningwang/work/ams/ams/cases/matpower/case14.m"...
Input file parsed in 0.0043 seconds.
Zero line rates detacted in rate_a, rate_b, rate_c, adjusted to 999.
System set up in 0.0024 seconds.


In [5]:
sp.DCOPF.run(solver='CLARABEL')

Building system matrices
Parsing OModel for <DCOPF>
Evaluating OModel for <DCOPF>
Finalizing OModel for <DCOPF>
<DCOPF> initialized in 0.0374 seconds.
<DCOPF> solved as optimal in 0.0164 seconds, converged in 11 iterations with CLARABEL.


True

In [6]:
sp.DCOPF.pg.v.round(2)

array([2.21, 0.38, 0.  , 0.  , 0.  ])

Then, we can lower down the generator maximum output to create an unsolvable problem.

In [ ]:
sp.StaticGen.set(src='pmax', attr='v', idx=[1, 2, 3, 4, 5], value=0.5)

True

Remember to update the corresponding parameters in the routine.

In [8]:
sp.DCOPF.update('pmax')

True

Now, we can see that the problem is infeasible.

In [9]:
sp.DCOPF.run(solver='CLARABEL')

DCOPF failed as infeasible in 7 iterations with CLARABEL!


False

Then, we can turn off some constraints and sovle it again.

In [10]:
sp.DCOPF.disable(['pglb', 'pgub'])

Turn off constraints: pglb, pgub


True

Note that since there are some constraints touched, the OModel will be updated automatically.

In [11]:
sp.DCOPF.run(solver='CLARABEL')

Disabled constraints: pglb, pgub
Finalizing OModel for <DCOPF>
<DCOPF> initialized in 0.0021 seconds.
<DCOPF> solved as optimal in 0.0062 seconds, converged in 9 iterations with CLARABEL.


True

In [12]:
sp.DCOPF.pg.v

array([ 2.31448356,  0.39836206, -0.04094854, -0.04094854, -0.04094854])

In [13]:
sp.DCOPF.pgub.e

array([ 2.01448356,  0.09836206, -0.34094854, -0.34094854, -0.34094854])

In [14]:
sp.DCOPF.pglb.e

array([-2.31448356, -0.39836206,  0.04094854,  0.04094854,  0.04094854])